<a href="https://colab.research.google.com/github/rkat7/practice-cuda/blob/main/starting_cuda_on_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pwd
%cd drive
#applying the cd to the entire notebook using %
%cd MyDrive
%mkdir practice-cuda
%cd practice-cuda
%pwd



/content/drive/MyDrive
[Errno 2] No such file or directory: 'drive'
/content/drive/MyDrive
[Errno 2] No such file or directory: 'MyDrive'
/content/drive/MyDrive
/content/drive/MyDrive/practice-cuda


'/content/drive/MyDrive/practice-cuda'

In [ ]:
!which nvcc #checking if cuda compiler is installed/present in the environment
!ls -l /dev/nv* #Checking if an Nvidia gpu is mounted to current colab environment

/usr/local/cuda/bin/nvcc
crw-rw-rw- 1 root root 195,   0 Feb 12 03:13 /dev/nvidia0
crw-rw-rw- 1 root root 195, 255 Feb 12 03:13 /dev/nvidiactl
crw-rw-rw- 1 root root 241,   0 Feb 12 03:13 /dev/nvidia-uvm
crw-rw-rw- 1 root root 241,   1 Feb 12 03:13 /dev/nvidia-uvm-tools

/dev/nvidia-caps:
total 0
cr-------- 1 root root 244, 1 Feb 12 03:13 nvidia-cap1
cr--r--r-- 1 root root 244, 2 Feb 12 03:13 nvidia-cap2


In [ ]:
!nvcc test.cu -o test

In [ ]:
!./test

Hello from CPU! 


In [1]:
!nvidia-smi

Mon Sep 15 04:27:19 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   55C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
# CUDA device global memory management APIs:

#cudaMalloc()
#cudaFree()

%%writefile device_mem_sample.cu
#include <stdio.h>

int main(){

float *A_d;
int size = 4 * sizeof(float);

cudaMalloc((void**) &A_d, size);

//some computations
cudaFree(A_d);

return 0;
}


Overwriting device_mem_sample.cu


In [5]:
!nvcc device_mem_sample.cu -o device_mem_sample

In [7]:
!./device_mem_sample